In [99]:
import pandas as pd
import json
import re
#from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
eval_name = "eval_make_model_aids"
eval_name = "eval_make_model_wildlife"
eval_name = "eval_make_model_bts"
model_name = "make_model_union"
run_name = eval_name + "_model_" + model_name

# load JSONL
records = []
with open("../aircraft_er_predictions/" + run_name + "_predictions_all.tsv") as f:
    for line in f:
        records.append(json.loads(line))



In [101]:
df = pd.DataFrame(records)
print(len(df))
df.head()

16755


,left,right,match,match_confidence
0,COL make VAL AIRBUS COL model VAL A300 COL nam...,COL aircraft VAL A-220\n,0,0.999599
1,COL make VAL AIRBUS COL model VAL A310 COL nam...,COL aircraft VAL A-220\n,0,0.999839
2,COL make VAL AIRBUS COL model VAL A318,COL aircraft VAL A-220\n,0,0.995111
3,COL make VAL AIRBUS COL model VAL A319,COL aircraft VAL A-220\n,0,0.996471
4,COL make VAL AIRBUS COL model VAL A320 COL nam...,COL aircraft VAL A-220\n,0,0.999993


In [102]:
df_raw = pd.read_csv("../data/ditto_aircraft/" + eval_name + "/all_pairs_with_id.txt", sep="," )
print(len(df_raw))
df_raw.head()

16755


,left_id,right_id,left,right
0,AIRBUS A300,A-220,COL make VAL AIRBUS COL model VAL A300 COL nam...,COL aircraft VAL A-220
1,AIRBUS A310,A-220,COL make VAL AIRBUS COL model VAL A310 COL nam...,COL aircraft VAL A-220
2,AIRBUS A318,A-220,COL make VAL AIRBUS COL model VAL A318,COL aircraft VAL A-220
3,AIRBUS A319,A-220,COL make VAL AIRBUS COL model VAL A319,COL aircraft VAL A-220
4,AIRBUS A320,A-220,COL make VAL AIRBUS COL model VAL A320 COL nam...,COL aircraft VAL A-220


In [103]:
df["left_id"] = df_raw["left_id"] 
df["right_id"] = df_raw["right_id"] 


In [104]:
df.head()

,left,right,match,match_confidence,left_id,right_id
0,COL make VAL AIRBUS COL model VAL A300 COL nam...,COL aircraft VAL A-220\n,0,0.999599,AIRBUS A300,A-220
1,COL make VAL AIRBUS COL model VAL A310 COL nam...,COL aircraft VAL A-220\n,0,0.999839,AIRBUS A310,A-220
2,COL make VAL AIRBUS COL model VAL A318,COL aircraft VAL A-220\n,0,0.995111,AIRBUS A318,A-220
3,COL make VAL AIRBUS COL model VAL A319,COL aircraft VAL A-220\n,0,0.996471,AIRBUS A319,A-220
4,COL make VAL AIRBUS COL model VAL A320 COL nam...,COL aircraft VAL A-220\n,0,0.999993,AIRBUS A320,A-220


In [105]:
### ONLY FOR BTS
df["description"] = df["right"].str.extract(
    r"COL\s+description\s+VAL\s*(.*?)(?=\n\s*COL|$)",
    expand=False
)

In [106]:

PAIR_RE = re.compile(r"COL\s+(?P<key>.*?)\s+VAL\s+(?P<val>.*?)(?=\s+COL\s+|$)")

def record_to_kv_list(record: str):
    """Return list like ['make: WACO', 'model: EGC', ...] in original order."""
    s = str(record)
    return [f"{m.group('key').strip()}: {m.group('val').strip()}"
            for m in PAIR_RE.finditer(s)]

In [107]:
# assuming your dataframe is called matches and has columns "left" and "right"
df["left_kv"]  = df["left"].apply(record_to_kv_list)
df["right_kv"] = df["right"].apply(record_to_kv_list)

In [108]:
df_agg = df.groupby("left_id", as_index=False).agg(
    left_count=("left", "count"),
    right_kv_distinct=("right_kv", lambda s: s.dropna().apply(tuple).nunique()),
)

In [109]:
df_joined = df.merge(
    df_agg,
    how="left",
    on=["left_id"],
)

In [110]:
df_joined

,left,right,match,match_confidence,left_id,right_id,description,left_kv,right_kv,left_count,right_kv_distinct
0,COL make VAL AIRBUS COL model VAL A300 COL nam...,COL aircraft VAL A-220\n,0,0.999599,AIRBUS A300,A-220,NaN,"[make: AIRBUS, model: A300, name: AIRBUS]",[aircraft: A-220],12,12
1,COL make VAL AIRBUS COL model VAL A310 COL nam...,COL aircraft VAL A-220\n,0,0.999839,AIRBUS A310,A-220,NaN,"[make: AIRBUS, model: A310, name: AIRBUS]",[aircraft: A-220],12,12
2,COL make VAL AIRBUS COL model VAL A318,COL aircraft VAL A-220\n,0,0.995111,AIRBUS A318,A-220,NaN,"[make: AIRBUS, model: A318]",[aircraft: A-220],12,12
3,COL make VAL AIRBUS COL model VAL A319,COL aircraft VAL A-220\n,0,0.996471,AIRBUS A319,A-220,NaN,"[make: AIRBUS, model: A319]",[aircraft: A-220],12,12
4,COL make VAL AIRBUS COL model VAL A320 COL nam...,COL aircraft VAL A-220\n,0,0.999993,AIRBUS A320,A-220,NaN,"[make: AIRBUS, model: A320, name: AIRBUS]",[aircraft: A-220],12,12
...,...,...,...,...,...,...,...,...,...,...,...
16750,COL make VAL MARTIN COL model VAL B26,COL aircraft VAL WB-57\n,0,0.983039,MARTIN B26,WB-57,NaN,"[make: MARTIN, model: B26]",[aircraft: WB-57],1,1
16751,COL make VAL WEATHERLY COL model VAL 201,COL aircraft VAL WEATHERLY 620\n,0,0.999704,WEATHERLY 201,WEATHERLY 620,NaN,"[make: WEATHERLY, model: 201]",[aircraft: WEATHERLY 620],1,1
16752,COL make VAL WEATHERLY COL model VAL 620,COL aircraft VAL WEATHERLY 620\n,1,0.999356,WEATHERLY 620,WEATHERLY 620,NaN,"[make: WEATHERLY, model: 620]",[aircraft: WEATHERLY 620],1,1
16753,COL make VAL ZENAIR COL model VAL CH2000,COL aircraft VAL ZODIAC CH601\n,0,0.999914,ZENAIR CH2000,ZODIAC CH601,NaN,"[make: ZENAIR, model: CH2000]",[aircraft: ZODIAC CH601],1,1


In [111]:
print('all evaluated potential paris: ', len(df))
df_matches = df[df["match"]==1]
print('matching paris: ', len(df_matches))

all evaluated potential paris:  16755
matching paris:  651


In [112]:
df_matches.head()

,left,right,match,match_confidence,left_id,right_id,description,left_kv,right_kv
5,COL make VAL AIRBUS COL model VAL A321,COL aircraft VAL A-220\n,1,0.538333,AIRBUS A321,A-220,NaN,"[make: AIRBUS, model: A321]",[aircraft: A-220]
9,COL make VAL AIRBUS COL model VAL A300 COL nam...,COL aircraft VAL A-300\n,1,0.970889,AIRBUS A300,A-300,NaN,"[make: AIRBUS, model: A300, name: AIRBUS]",[aircraft: A-300]
14,COL make VAL AIRBUS COL model VAL A321,COL aircraft VAL A-300\n,1,0.372120,AIRBUS A321,A-300,NaN,"[make: AIRBUS, model: A321]",[aircraft: A-300]
32,COL make VAL AIRBUS COL model VAL A321,COL aircraft VAL A-318\n,1,0.585662,AIRBUS A321,A-318,NaN,"[make: AIRBUS, model: A321]",[aircraft: A-318]
38,COL make VAL AIRBUS COL model VAL A318,COL aircraft VAL A-319\n,1,0.713871,AIRBUS A318,A-319,NaN,"[make: AIRBUS, model: A318]",[aircraft: A-319]


In [113]:
df_max = (
    df_matches
      .groupby(['right_id'], as_index=False)
      .agg({
          'match_confidence':      "max",
          'left_id':              lambda s: pd.unique(s).tolist(),
      })
      .rename(columns={'match_confidence': 'match_confidence_max', 'left_id': 'left_ids'})
      .assign(left_ids_count=lambda d: d['left_ids'].str.len())
)



In [114]:
df_max

,right_id,match_confidence_max,left_ids,left_ids_count
0,A-220,0.538333,[AIRBUS A321],1
1,A-300,0.970889,"[AIRBUS A300, AIRBUS A321]",2
2,A-318,0.585662,[AIRBUS A321],1
3,A-319,0.898061,"[AIRBUS A318, AIRBUS A319, AIRBUS A321]",3
4,A-320,0.998108,[AIRBUS A320],1
...,...,...,...,...
380,TECNAM P2010,0.999959,[TECNAM P2010],1
381,TECNAM P2012 TRAVELL,0.998921,[TECNAM P2012],1
382,UH-1H,0.999992,[BELL UH1],1
383,VULCANAIR V1.0,0.999143,[VULCANAIR V1],1


In [115]:
df_join = df_matches.merge(
    df_max,
    left_on="right_id",
    right_on="right_id",
    how='left'
)
df_join["is_max"] = df_join["match_confidence"]==df_join["match_confidence_max"]

In [116]:
df_join

,left,right,match,match_confidence,left_id,right_id,description,left_kv,right_kv,match_confidence_max,left_ids,left_ids_count,is_max
0,COL make VAL AIRBUS COL model VAL A321,COL aircraft VAL A-220\n,1,0.538333,AIRBUS A321,A-220,NaN,"[make: AIRBUS, model: A321]",[aircraft: A-220],0.538333,[AIRBUS A321],1,True
1,COL make VAL AIRBUS COL model VAL A300 COL nam...,COL aircraft VAL A-300\n,1,0.970889,AIRBUS A300,A-300,NaN,"[make: AIRBUS, model: A300, name: AIRBUS]",[aircraft: A-300],0.970889,"[AIRBUS A300, AIRBUS A321]",2,True
2,COL make VAL AIRBUS COL model VAL A321,COL aircraft VAL A-300\n,1,0.372120,AIRBUS A321,A-300,NaN,"[make: AIRBUS, model: A321]",[aircraft: A-300],0.970889,"[AIRBUS A300, AIRBUS A321]",2,False
3,COL make VAL AIRBUS COL model VAL A321,COL aircraft VAL A-318\n,1,0.585662,AIRBUS A321,A-318,NaN,"[make: AIRBUS, model: A321]",[aircraft: A-318],0.585662,[AIRBUS A321],1,True
4,COL make VAL AIRBUS COL model VAL A318,COL aircraft VAL A-319\n,1,0.713871,AIRBUS A318,A-319,NaN,"[make: AIRBUS, model: A318]",[aircraft: A-319],0.898061,"[AIRBUS A318, AIRBUS A319, AIRBUS A321]",3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,COL make VAL BELL COL model VAL UH1 COL name V...,COL aircraft VAL UH-1H\n,1,0.999992,BELL UH1,UH-1H,NaN,"[make: BELL, model: UH1, name: BELL204]",[aircraft: UH-1H],0.999992,[BELL UH1],1,False
647,COL make VAL BELL COL model VAL UH1 COL name V...,COL aircraft VAL UH-1H\n,1,0.999992,BELL UH1,UH-1H,NaN,"[make: BELL, model: UH1, name: BELL204 BELL205]",[aircraft: UH-1H],0.999992,[BELL UH1],1,False
648,COL make VAL BELL COL model VAL UH1 COL name V...,COL aircraft VAL UH-1H\n,1,0.999992,BELL UH1,UH-1H,NaN,"[make: BELL, model: UH1, name: BELL 205]",[aircraft: UH-1H],0.999992,[BELL UH1],1,False
649,COL make VAL VULCANAIR COL model VAL V1,COL aircraft VAL VULCANAIR V1.0\n,1,0.999143,VULCANAIR V1,VULCANAIR V1.0,NaN,"[make: VULCANAIR, model: V1]",[aircraft: VULCANAIR V1.0],0.999143,[VULCANAIR V1],1,True


In [117]:
df_join.to_csv("../aircraft_er_predictions/" + run_name + "_all_matches.csv", index=False)

In [118]:
df_join_max = df_join[df_join["is_max"]==True]

df_join_max

,left,right,match,match_confidence,left_id,right_id,description,left_kv,right_kv,match_confidence_max,left_ids,left_ids_count,is_max
0,COL make VAL AIRBUS COL model VAL A321,COL aircraft VAL A-220\n,1,0.538333,AIRBUS A321,A-220,NaN,"[make: AIRBUS, model: A321]",[aircraft: A-220],0.538333,[AIRBUS A321],1,True
1,COL make VAL AIRBUS COL model VAL A300 COL nam...,COL aircraft VAL A-300\n,1,0.970889,AIRBUS A300,A-300,NaN,"[make: AIRBUS, model: A300, name: AIRBUS]",[aircraft: A-300],0.970889,"[AIRBUS A300, AIRBUS A321]",2,True
3,COL make VAL AIRBUS COL model VAL A321,COL aircraft VAL A-318\n,1,0.585662,AIRBUS A321,A-318,NaN,"[make: AIRBUS, model: A321]",[aircraft: A-318],0.585662,[AIRBUS A321],1,True
5,COL make VAL AIRBUS COL model VAL A319,COL aircraft VAL A-319\n,1,0.898061,AIRBUS A319,A-319,NaN,"[make: AIRBUS, model: A319]",[aircraft: A-319],0.898061,"[AIRBUS A318, AIRBUS A319, AIRBUS A321]",3,True
7,COL make VAL AIRBUS COL model VAL A320 COL nam...,COL aircraft VAL A-320\n,1,0.998108,AIRBUS A320,A-320,NaN,"[make: AIRBUS, model: A320, name: AIRBUS]",[aircraft: A-320],0.998108,[AIRBUS A320],1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,COL make VAL TECNAM COL model VAL P2010,COL aircraft VAL TECNAM P2010\n,1,0.999959,TECNAM P2010,TECNAM P2010,NaN,"[make: TECNAM, model: P2010]",[aircraft: TECNAM P2010],0.999959,[TECNAM P2010],1,True
622,COL make VAL TECNAM COL model VAL P2012,COL aircraft VAL TECNAM P2012 TRAVELL\n,1,0.998921,TECNAM P2012,TECNAM P2012 TRAVELL,NaN,"[make: TECNAM, model: P2012]",[aircraft: TECNAM P2012 TRAVELL],0.998921,[TECNAM P2012],1,True
644,COL make VAL BELL COL model VAL UH1 COL name V...,COL aircraft VAL UH-1H\n,1,0.999992,BELL UH1,UH-1H,NaN,"[make: BELL, model: UH1, name: BELL205]",[aircraft: UH-1H],0.999992,[BELL UH1],1,True
649,COL make VAL VULCANAIR COL model VAL V1,COL aircraft VAL VULCANAIR V1.0\n,1,0.999143,VULCANAIR V1,VULCANAIR V1.0,NaN,"[make: VULCANAIR, model: V1]",[aircraft: VULCANAIR V1.0],0.999143,[VULCANAIR V1],1,True


In [119]:
df_join_max.to_csv('temp.csv')

df_join_max.to_csv("../aircraft_er_predictions/" + run_name + "_max_matches.csv", index=False)

In [120]:
df_matches[df_matches["right"].isna()]


,left,right,match,match_confidence,left_id,right_id,description,left_kv,right_kv
